## 2024-02-12

Stephanie Leroux

This notebook summarizes my thoughts about Hugo Lecomte's question from his 2024-02-12 email about recovering mass fluctuations from the bottom pressure model output and compare mass fluctuations recoved from the SSH model oputput.

---

I'm downloading first a few python packages for later test purposes:

In [57]:
## standart libraries
import os,sys
import numpy as np

# xarray
import xarray as xr

# time
import datetime
import netCDF4
import cftime


# for jupyter notebook display
%matplotlib inline

# 1. Bottom pressure  in the model

* In the model, the density $\rho$ is described as: $\rho=\rho_0(1+rhd)$ where $\rho_0=1026$ kg.m-3.

* The bottom pressure $p_b$ at each bottom grid cell is the pressure of the above column of liquid ocean:

$p_b=\int_{-H}^{\eta(t)}\rho g dz$

where $H$ is the mean thickness of the ocean at rest and $\eta(t)$ is the heigh of the time-varying free surface above 0, so that the total heigh of the liquid column at time t is $H+\eta(t)$. $g$ is the gravity, taken as constant in the model: $g$=9.8065 m.s-2

* From the above, $p_b$ can be expended to: 

$
p_b=\int_{-H}^{\eta(t)}\rho g dz \\
\quad= g\int_{-H}^{\eta(t)}\rho_0(1+rhd)\times dz\\
\quad= g\rho_0(H+\eta(t))+g\rho_0\int_{-H}^{\eta(t)}rhd\times dz
$

* Given the vertical discretization used in the model (`vvl` option), the vertical metrics adjust to the time-varying free surface height. The discretized $dz$ (height of each cell) varies in time. In the model it is named $e3_T(t)$. So in the discretized world, the above expression for $p_b$ can express as: 

$
p_b=g\rho_0\biggl(H+\eta(t)+\sum_{jpkm1}^{1}rhd(t)\times e3_T(t)\biggr)
$

* It seems to me that the  expression above is consistent with how the variable `botpres` is computed in the model code, in subroutine `diaar5.F90`:

```fortran
zbotpres(:,:) = 0._wp                        
[...]
DO jk = 1, jpkm1
 zbotpres(:,:) = zbotpres(:,:) + e3t_n(:,:,jk) * rhd(:,:,jk)
END DO
[...]
zztmp = rau0 * grav * 1.e-4_wp                             
zbotpres(:,:) = zztmp * ( zbotpres(:,:) + sshn(:,:) + thick0(:,:) )
```

Note: the 1.e-4  factor comes from a conversion to Pa to dbar which is the unit used for `botpres` in the model outputs.

---

# 2. From bottom pressure to mass

Bottom pressure is a pressure, i.e.  a force by area unit. 

To convert to mass, you need to multiply $p_b$ by the area of each cell (which gives you the weight) and then divide by $g$ to derive the mass of each ocean column. 

Then to derive the mass of the entire global liquid ocean ($M_{go}$), you need to sum over all ocean grid cells:

$
M_{go}=\dfrac{1}{g}\sum_{allcells}\biggl(p_b\times e1_T.e2_T\biggr)
$

### 2.1 Sanity check of the bottom pressure variable in the model outputs:

* As a quick sanity check, we can compute $M_{go}$ from the model outputs:

In [65]:
diri="/Users/leroux/Downloads/"
fili="mesh_hgr.nc"
filibp="TM_1993-2018_OBP_eORCA025.L75-IMHOTEP.GAIc_1d.nc"

# read grid infos
datagrid = xr.open_dataset(diri+fili,decode_times=False)

# read time-mean bottom pressure
obp = xr.open_dataset(diri+filibp,decode_times=False)["botpres"]

* Compute area of each model grid cell:

In [67]:
AC = datagrid.e1t.squeeze()*datagrid.e2t.squeeze()

* Convert pressure to weight by multiplying OBP by the area:

In [68]:
# in Pa then multiplied by surface of each cell:
Weight = obp.squeeze()*10000*AC

* Convert Weight to Mass by dividing by g and sum over all cells

In [69]:
# mass
g = 9.8065

Mass = (1/g)*Weight.sum()

# Mass of the ocean in kg: (should be around 1.4e21 kg
Mass

<xarray.DataArray ()>
array(1.36294809e+21)

--> you get the right kind of number for the averaged mass of the ocean (around 1.4e21 kg from wikipedia)

---

### 2.2 Now expend $M_{go}$ to relate to SSH ($\eta$)

You can expend $M_{go}$ using the expression of $p_b$ above:

$
M_{go}=\dfrac{g\rho_0 }{g}\sum_{allcells}\biggl(H+\eta(t)\times e1_T.e2_T\biggr)  + \dfrac{g\rho_0 }{g}\sum_{allcells}\Biggl(\sum_{jpk-1}^{1}\biggl(rhd\times e3_T(t)\biggr) \times e1_T.e2_T\Biggr)
$

For visual simplification, let's note $\biggl<...\biggl>$ the spatial  average operator over all the ocean grid cells $\sum_{allcells}\biggl(...\biggl) \times e1_T.e2_T$

$
M_{go}=\rho_0 \biggl<H+\eta(t)\biggr>  + \rho_0 \Biggl<\sum_{jpk-1}^{1}\biggl(rhd(t)\times e3_T(t)\biggr) \Biggr>
$

### 2.3 Link with Hugo's question

Hugo wants to compare the fluctuations in time of the global mass. So let's call those flutuations $M'_{go} = M_{go}-\overline{M_{go}}$

Now let's examine which terms can vary in time in the expression of $M_{go}$ above:

* the term $\rho_0\biggl<H\biggl>$  is constant with time, so it won't remain in $M_{go}'$.

* the term $\rho_0\biggl<\eta(t)\biggl>$ varies in time because of freshwater fluxes. Recall that in the model, E-P+R+fwfcorr is reset to zero at each time-step, but the sea ice annual cycle is free. This is why, as we know already, the global mean of the SSH $\biggl<\eta(t)\biggl>$ is not exactly zero: its small fluctuations are due to the freezing/melting of sea ice that makes the total mount of liquid water in the model fluctuate in time. Or in other words, the mass of ocean liquid water is not conserved at each time step, since part of it can be temporarily in its solid form.

* Now how about the last term in $M_{go}$, that we will call  $\mathcal{D}=\rho_0 \Biggl<\sum_{jpk-1}^{1}\biggl(rhd(t)\times e3_T(t)\biggr) \Biggr>$ ?

Hugo hypothetised that its time mean $\overline{\mathcal{D}}$ is null because the model is Businesq and in average over the globe, the mass fluctuations only come from the external fluxes.

**_If_** it is true that $\overline{\mathcal{D}}=0$ then the only varying term in  $M_{go}$ is the one with $\eta(t)$, so:

$M'_{go}=\rho_0\biggl<\eta(t)\biggl>$. 

In that case, we should indeed get $M'_{go}$ computed from the OBP model outputs equals  $\rho_0\biggl<\eta(t)\biggl>'$ computed from the model output SSH (the prime denotes that the time mean was removed). In other words, Hugo's blue curve and orange curve should be superposed.

BUT - we find that it is not exactly the case: we get a residual that looks like a seasonal cycle. Why?

### 2.4 My understanding so far...

If the blue and orange curves are not stricly superposed, it must mean that our hypothetis $\overline{\mathcal{D}}=0$ is not exactly true. Why?

My guess is that it is true only in average over 1 year, but not at the daily frequency. The fact that the bottom pressure is computed from the liquid ocean only might be the explanation. Between winter and summer, we might have not only small variations in the amount of liquid water (due to freezing/melting ice, and asymetry between the 2 hemispheres) but also, as a consequence, small variations in the relative density, which impact the global mean term $\mathcal{D}=\rho_0 \Biggl<\sum_{jpk-1}^{1}\biggl(rhd(t)\times e3_T(t)\biggr) \Biggr>$. And again, there is also  $e3T(t)$ that fluctuates.

At this stage i am not sure how we could verify this (that $\overline{\mathcal{D}}$ is not strictly zero$), but it seems to me that it must be the explanation why we get this little difference between the orange and blue curves.

We could try to recompute the $\overline{\mathcal{D}}$ term from the T and S 3-D model fields maybe, if necessary...


